# Baseline

# Config

## Library

In [5]:
from datasets import load_dataset

In [7]:
raw_dataset = load_dataset("jinmang2/load_klue_re", script_version="v1.0.1b", cache_dir="cache")

Reusing dataset klue_re (cache/klue_re/re/1.0.1/72db5b4f9111ca3106d23ecec67a53f750c70b70b32c5512f925baac3b39a0de)


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
df_new = pd.read_csv('submission-baseline-0929-roberta-large-warmup0.2_lr3e-5-test.csv')
df_old = pd.read_csv('submission.csv')

In [14]:
(df_new.pred_label == df_old.pred_label).sum() / len(df_new)

0.8520283322601416

In [4]:
import pickle as pickle
import os
from functools import partial
from typing import Tuple, List, Any, Dict
import random
import pandas as pd
import torch
import sklearn
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer

print(os.getcwd())

# from metrics import *
# from load_data import *

/opt/ml/klue_re


In [15]:
import torch

## Settings

In [2]:
##----------------
def set_seeds(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # for faster training, but not deterministic

set_seeds()    

In [3]:
class_names = ['no_relation', 'org:top_members/employees', 'org:members',
       'org:product', 'per:title', 'org:alternate_names',
       'per:employee_of', 'org:place_of_headquarters', 'per:product',
       'org:number_of_employees/members', 'per:children',
       'per:place_of_residence', 'per:alternate_names',
       'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings',
       'per:spouse', 'org:founded', 'org:political/religious_affiliation',
       'org:member_of', 'per:parents', 'org:dissolved',
       'per:schools_attended', 'per:date_of_death', 'per:date_of_birth',
       'per:place_of_birth', 'per:place_of_death', 'org:founded_by',
       'per:religion']

id2label = {idx: label for idx, label in enumerate(class_names)}
label2id = {label: idx for idx, label in enumerate(class_names)}

KLUE에서 제시된 hyperparameter setting 

### Fine-tuning

In [4]:
# Search space
# tune_config = {
#     "per_device_train_batch_size": tune.choice([8, 16, 32]),
#     "per_device_eval_batch_size": 32,
#     "learning_rate": tune.choice([1e-5, 2e-5, 3e-5, 5e-5]),
#     "warmup_ratio": tune.choice([0., 0.1, 0.2, 0.6]),
#     "weihgt_decay": tune.choice([0.0, 0.01]),
#     "num_train_epochs": tune.choice([3, 4, 5, 10]),
# }

In [5]:
# patience = 10000
output_dir = "klue_dir/klue-roberta-large-warmup0.2_3e-5-newdata"
wandb_project = 'klue_re_klue-roberta-large-daeug'
run_name = 'baseline-0928-roberta-large-warmup0.2_lr3e-5-newdata'
report_to = "wandb"
max_length = 128
cache_dir = 'cache'

MODEL_NAME = "klue/roberta-large"

In [6]:
import datasets

In [7]:
# Version 1.0.1 - train test split
klue_re = datasets.load_dataset("jinmang2/load_klue_re",script_version="v1.0.1", cache_dir='cache')

Downloading:   0%|          | 0.00/6.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.15M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset klue_re downloaded and prepared to cache/klue_re/re/1.0.0/5edf8095b936bfcc409e00bd241e5f6dad3b4c6e21e1df9f42be7e4c1eb3918c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Data

## Load dataset

In [9]:
dataset = pd.read_csv("../dataset/train/train.csv")
label = label_to_num(dataset['label'].values)

dataset['id'] = dataset['id'].astype(str)
dataset['subject_entity'] = dataset['subject_entity'].apply(lambda x : eval(x))
dataset['object_entity'] = dataset['object_entity'].apply(lambda x : eval(x))
dataset['label'] = dataset['label'].map(label2id)

with open('train_eval_idx.pkl', 'rb') as f:
    train_eval_idx = pickle.load(f)
    
train_dataset = dataset.iloc[train_eval_idx['train']]
dev_dataset = dataset.iloc[train_eval_idx['eval']]

train_label = np.array(label)[train_eval_idx['train']]
dev_label = np.array(label)[train_eval_idx['eval']]

In [10]:
from datasets import Dataset, DatasetDict, Features, Value, ClassLabel
class_labels = ClassLabel(num_classes=30, names=class_names)

hf_train_dataset = Dataset.from_pandas(train_dataset)
hf_dev_dataset = Dataset.from_pandas(dev_dataset)

hf_train_dataset = hf_train_dataset.remove_columns('__index_level_0__')
hf_dev_dataset = hf_dev_dataset.remove_columns('__index_level_0__')

klue_re = DatasetDict({'train' : hf_train_dataset, 'dev' : hf_dev_dataset})

klue_re['train'].features['label'] = ClassLabel(num_classes=30, names=class_names)
klue_re['dev'].features['label'] = ClassLabel(num_classes=30, names=class_names)

In [11]:
klue_re['train'].features

{'id': Value(dtype='string', id=None),
 'sentence': Value(dtype='string', id=None),
 'subject_entity': {'end_idx': Value(dtype='int64', id=None),
  'start_idx': Value(dtype='int64', id=None),
  'type': Value(dtype='string', id=None),
  'word': Value(dtype='string', id=None)},
 'object_entity': {'end_idx': Value(dtype='int64', id=None),
  'start_idx': Value(dtype='int64', id=None),
  'type': Value(dtype='string', id=None),
  'word': Value(dtype='string', id=None)},
 'label': ClassLabel(num_classes=30, names=['no_relation', 'org:top_members/employees', 'org:members', 'org:product', 'per:title', 'org:alternate_names', 'per:employee_of', 'org:place_of_headquarters', 'per:product', 'org:number_of_employees/members', 'per:children', 'per:place_of_residence', 'per:alternate_names', 'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings', 'per:spouse', 'org:founded', 'org:political/religious_affiliation', 'org:member_of', 'per:parents', 'org:dissolved', 'per:schools_attended', 'per:

In [12]:
hf_train_dataset[0]

{'id': '10706',
 'sentence': '추승우(秋承佑, 1979년 9월 24일 ~)는 전 KBO 리그 한화 이글스의 외야수이자, 현재 KBO 리그 한화 이글스의 작전·주루 및 외야수 수비 코치이다.',
 'subject_entity': {'end_idx': 2,
  'start_idx': 0,
  'type': 'PER',
  'word': '추승우'},
 'object_entity': {'end_idx': 78,
  'start_idx': 76,
  'type': 'POH',
  'word': '외야수'},
 'label': 0,
 'source': 'wikipedia'}

## Tokenizing

In [13]:
# tokenizing dataset
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=cache_dir)

/opt/conda/envs/basic/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [14]:
# create_example

markers = dict(
    subject_start_marker="<subj>",
    subject_end_marker="</subj>",
    object_start_marker="<obj>",
    object_end_marker="</obj>",
)

def mark_entity_spans(examples,
                      subject_start_marker: str, subject_end_marker: str,
                      object_start_marker: str, object_end_marker: str):

    def _mark_entity_spans(
        text: str, 
        subject_range=Tuple[int, int], 
        object_range=Tuple[int, int]
    ) -> str:
        """ Adds entity markers to the text to identify the subject/object entities.
        Args:
            text: Original sentence
            subject_range: Pair of start and end indices of subject entity
            object_range: Pair of start and end indices of object entity
        Returns:
            A string of text with subject/object entity markers
        """
        if subject_range < object_range:
            segments = [
                text[: subject_range[0]],
                subject_start_marker,
                text[subject_range[0] : subject_range[1] + 1],
                subject_end_marker,
                text[subject_range[1] + 1 : object_range[0]],
                object_start_marker,
                text[object_range[0] : object_range[1] + 1],
                object_end_marker,
                text[object_range[1] + 1 :],
            ]
        elif subject_range > object_range:
            segments = [
                text[: object_range[0]],
                object_start_marker,
                text[object_range[0] : object_range[1] + 1],
                object_end_marker,
                text[object_range[1] + 1 : subject_range[0]],
                subject_start_marker,
                text[subject_range[0] : subject_range[1] + 1],
                subject_end_marker,
                text[subject_range[1] + 1 :],
            ]
        else:
            raise ValueError("Entity boundaries overlap.")

        marked_text = "".join(segments)

        return marked_text
    
    subject_entity = examples["subject_entity"]
    object_entity = examples["object_entity"]
    
    text = _mark_entity_spans(
        examples["sentence"],
        (subject_entity["start_idx"], subject_entity["end_idx"]),
        (object_entity["start_idx"], object_entity["end_idx"]),
    )
    return {"text": text}

mark_entity_spans = partial(mark_entity_spans, **markers)

In [15]:
examples = klue_re.map(mark_entity_spans)

  0%|          | 0/25976 [00:00<?, ?ex/s]

  0%|          | 0/6494 [00:00<?, ?ex/s]

In [16]:
tokenizer.add_special_tokens(
    {"additional_special_tokens": list(markers.values())}
)

4

In [17]:
def convert_example_to_features(
    examples, 
    tokenizer,
    subject_start_marker: str,
    subject_end_marker: str,
    object_start_marker: str,
    object_end_marker: str
) -> Dict[str, List[Any]]:
    
    def fix_tokenization_error(text: str) -> List[str]:
        """Fix the tokenization due to the `obj` and `subj` marker inserted
        in the middle of a word.
        Example:
            >>> text = "<obj>조지 해리슨</obj>이 쓰고 <subj>비틀즈</subj>가"
            >>> tokens = ['<obj>', '조지', '해리', '##슨', '</obj>', '이', '쓰', '##고', '<subj>', '비틀즈', '</subj>', '가']
            >>> fix_tokenization_error(text)
            ['<obj>', '조지', '해리', '##슨', '</obj>', '##이', '쓰', '##고', '<subj>', '비틀즈', '</subj>', '##가']
            
        Only support for BertTokenizerFast
        If you use bbpe, change code!
        """
        batch_encoding = tokenizer._tokenizer.encode(text)
        tokens = batch_encoding.tokens
        # subject
        if text[text.find(subject_end_marker) + len(subject_end_marker)] != " ":
            space_idx = tokens.index(subject_end_marker) + 1
            # tokenizer_type == "bert-wp"
            if not tokens[space_idx].startswith("##") and "가" <= tokens[space_idx][0] <= "힣":
                tokens[space_idx] = "##" + tokens[space_idx]

        # object
        if text[text.find(object_end_marker) + len(object_end_marker)] != " ":
            space_idx = tokens.index(object_end_marker) + 1
            # tokenizer_type == "bert-wp"
            if not tokens[space_idx].startswith("##") and "가" <= tokens[space_idx][0] <= "힣":
                tokens[space_idx] = "##" + tokens[space_idx]
        
        return tokens    
    
    tokens = fix_tokenization_error(examples["text"])
    
    return {
        "input_ids": tokenizer.convert_tokens_to_ids(tokens),
        "tokenized": tokens,
    }

convert_example_to_features = partial(
    convert_example_to_features,
    tokenizer=tokenizer,
    **markers,
)

In [18]:
tokenized_datasets = examples.map(convert_example_to_features)

  0%|          | 0/25976 [00:00<?, ?ex/s]

  0%|          | 0/6494 [00:00<?, ?ex/s]

# Model

In [19]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [20]:
# setting model hyperparameter
model_config =  AutoConfig.from_pretrained(MODEL_NAME)
model_config.num_labels = 30
model_config.cache_dir = 'cache'
model_config.id2label = id2label
model_config.label2id = label2id
# model_config.max_length = 128

model =  AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,
                                                            config=model_config,
                                                            cache_dir='cache')
print(model.config)
model.parameters
model.to(device)

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classif

RobertaConfig {
  "_name_or_path": "klue/roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "cache_dir": "cache",
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "no_relation",
    "1": "org:top_members/employees",
    "2": "org:members",
    "3": "org:product",
    "4": "per:title",
    "5": "org:alternate_names",
    "6": "per:employee_of",
    "7": "org:place_of_headquarters",
    "8": "per:product",
    "9": "org:number_of_employees/members",
    "10": "per:children",
    "11": "per:place_of_residence",
    "12": "per:alternate_names",
    "13": "per:other_family",
    "14": "per:colleagues",
    "15": "per:origin",
    "16": "per:siblings",
    "17": "per:spouse",
    "18": "org:founded",
    "19": "org:political/religious_affiliation",
    "20": "org:member_of",
   

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [21]:
# Vocab Resize
if model.config.vocab_size < len(tokenizer):
    print("resize...")
    model.resize_token_embeddings(len(tokenizer))

resize...


## Data Collator

In [22]:
class DataCollator:
    
    def __init__(self, tokenizer, max_length='single'):
        self.tokenizer = tokenizer
        if max_length == 'single':
            self.max_length = tokenizer.max_len_single_sentence
        elif max_length == 'pair':
            self.max_length = tokenizer.max_len_sentences_pair
        elif type(max_length) == int:
            self.max_length = max_length
    
    def __call__(self, batch):
        input_ids = [x["input_ids"] for x in batch]
        labels = [x["label"] for x in batch]
        batch_encoding = tokenizer.pad(
            {"input_ids": input_ids},
            max_length=self.max_length,
            return_tensors="pt",
        )
        batch_encoding.update({"labels": torch.LongTensor(labels)})
        return batch_encoding

In [23]:
data_collator = DataCollator(tokenizer, max_length=max_length)

In [24]:
os.environ["WANDB_PROJECT"] = wandb_project


call_wandb = True
try:
    os.environ["WANDB_PROJECT"]
    
except KeyError:
    call_wandb = False
    
if call_wandb:
    import wandb
    wandb.login()

wandb: Currently logged in as: kiyoung2 (use `wandb login --relogin` to force relogin)


In [25]:
training_args = TrainingArguments(
            
            # Checkpoint
            output_dir=output_dir,          # output directory
#             save_total_limit=5,              # number of total save model.
            save_strategy="epoch",
#             save_steps=500,                 # model saving step.
            
            # Training
            num_train_epochs=4,              # total number of training epochs
            learning_rate=3e-5,               # learning_rate
            per_device_train_batch_size=32,  # batch size per device during training
            per_device_eval_batch_size=32,   # batch size for evaluation
            
            ## Learning rate scheduling
#             lr_scheduler_type = 'cosine',
#             warmup_steps=500,                # number of warmup steps for learning rate scheduler
            warmup_ratio = 0.2,


            ## Regularization
            weight_decay=0.01,               # strength of weight decay

            # Logging
            logging_dir='./logs',            # directory for storing logs
#             logging_steps=100,              # log saving step.
            report_to = 'wandb',
            
            # Evaluation
            metric_for_best_model = 'auprc',
            evaluation_strategy='epoch', # evaluation strategy to adopt during training
                                        # `no`: No evaluation during training.
                                        # `steps`: Evaluate every `eval_steps`.
                                        # `epoch`: Evaluate every end of epoch.
            eval_steps = 500,            # evaluation step.
            
            # ETC    
            load_best_model_at_end = True,
            seed = 42,
            
            # Hugging Face Hub upload
#             push_to_hub = True,
#             push_to_hub_model_id = f"{model_name}-finetuned-{task}",
    
            # AMP
            fp16 = True,
            )

In [26]:
features_name = list(tokenized_datasets["train"].features.keys())
features_name.pop(features_name.index("input_ids"))
features_name.pop(features_name.index("label"))
tokenized_datasets = tokenized_datasets.remove_columns(features_name)

In [27]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids'],
        num_rows: 25976
    })
    dev: Dataset({
        features: ['label', 'input_ids'],
        num_rows: 6494
    })
})

In [29]:
trainer = Trainer(
            model=model,                         # the instantiated 🤗 Transformers model to be trained
            args=training_args,                  # training arguments, defined above
            train_dataset=tokenized_datasets['train'],         # training dataset
            eval_dataset=tokenized_datasets['dev'],             # evaluation dataset
#             eval_dataset=tokenized_datasets['dev'],             # evaluation dataset
            data_collator=data_collator,
            compute_metrics=compute_metrics         # define metrics function
)

Using amp fp16 backend


# Loss & Metrics

- `metrics.py` 참고

# Training

In [32]:
import wandb
wandb.login()

True

In [33]:
id = wandb.util.generate_id()
print(id)

23ozl9fw


생성된 id를 붙여 넣으면 `wandb`를 사용할 수 있습니다.
> **\[참고\]** - project : 실험기록을 관리할 프로젝트 이름. 없을 시 입력받은 이름으로 생성, 여기선 예시로 klue로 설정
- entity : weights & biases 사용자명 또는 팀 이름
- id : 실험에 부여된 고유 아이디
- name : 실험에 부여한 이름
- resume : 실험을 재개할 떄, 실험에 부여한 고유 아이디를 입력

In [34]:
wandb.init(project=wandb_project, # 실험기록을 관리한 프로젝트 이름
           entity='kiyoung2',     # 사용자명 또는 팀 이름
           id='23ozl9fw',      # 실험에 부여된 고유 아이디
           name=run_name,         # 실험에 부여한 이름
          )


CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



In [35]:
trainer.train()

***** Running training *****
  Num examples = 25976
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3248
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/opt/conda/envs/basic/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


Epoch,Training Loss,Validation Loss,Micro F1,Auprc
1,1.678500,0.669914,0.798275,0.717739
2,0.531100,0.540100,0.844027,0.761848
3,0.376100,0.495992,0.859564,0.819069
4,0.179900,0.609471,0.860984,0.819533


/opt/conda/envs/basic/lib/python3.8/site-packages/transformers/trainer.py:1347: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
***** Running Evaluation *****
  Num examples = 6494
  Batch size = 32
Saving model checkpoint to klue_dir/klue-roberta-large-warmup0.2_3e-5-newdata/checkpoint-812
Configuration saved in klue_dir/klue-roberta-large-warmup0.2_3e-5-newdata/checkpoint-812/config.json
Model weights saved in klue_dir/klue-roberta-large-warmup0.2_3e-5-newdata/checkpoint-812/pytorch_model.bin
/opt/conda/envs/basic/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`

TrainOutput(global_step=3248, training_loss=0.5932091069338944, metrics={'train_runtime': 1305.1997, 'train_samples_per_second': 79.608, 'train_steps_per_second': 2.489, 'total_flos': 2.4605941785440736e+16, 'train_loss': 0.5932091069338944, 'epoch': 4.0})

In [31]:
wandb.finish()

eval/auprc,0.81436
eval/loss,0.61059
eval/micro_f1,0.85837
eval/runtime,50.4615
eval/samples_per_second,128.692
eval/steps_per_second,4.023
train/epoch,4.0
train/global_step,3248
train/learning_rate,0.0
train/loss,0.182
train/total_flos,2.46133251276528e+16


eval/auprc,▁▆██
eval/loss,█▃▁▄
eval/micro_f1,▁▆██
eval/runtime,█▁▆▇
eval/samples_per_second,▁█▃▂
eval/steps_per_second,▁█▄▂
train/epoch,▁▂▂▄▄▅▆▆▇██
train/global_step,▁▂▂▄▄▅▆▆▇██
train/learning_rate,▇█▆▄▃▁
train/loss,█▄▃▂▂▁
train/total_flos,▁


# Inference

In [30]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [31]:
device

device(type='cuda', index=0)

In [35]:
# setting model hyperparameter
# checkpoint = './klue_dir/checkpoint-3248/'
checkpoint = 'klue_dir/klue-roberta-large-warmup0.2_3e-5/'
model_config =  AutoConfig.from_pretrained(MODEL_NAME)
model_config.num_labels = 30
model_config.cache_dir = 'cache'
model_config.id2label = id2label
model_config.label2id = label2id
model_config.vocab_size = 32004
# model_config.max_length = 128

model =  AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                            config=model_config,
                                                            cache_dir='cache')
print(model.config)
model.parameters
model.to(device)

loading configuration file https://huggingface.co/klue/roberta-large/resolve/main/config.json from cache at /opt/ml/.cache/huggingface/transformers/571e05a2160c18c93365862223c4dae92bbd1b41464a4bd5f372ad703dba6097.ae5b7f8d8a28a3ff0b1560b4d08c6c3bd80f627288eee2024e02959dd60380d0
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.11.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}



OSError: Error no file named ['pytorch_model.bin', 'tf_model.h5', 'model.ckpt.index', 'flax_model.msgpack'] found in directory klue_dir/klue-roberta-large-warmup0.2_3e-5/ or `from_tf` and `from_flax` set to False.

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import torch.nn.functional as F

In [ ]:
dataset = pd.read_csv("../dataset/test/test_data.csv")
# label = label_to_num(dataset['label'].values)

dataset['id'] = dataset['id'].astype(str)
dataset['subject_entity'] = dataset['subject_entity'].apply(lambda x : eval(x))
dataset['object_entity'] = dataset['object_entity'].apply(lambda x : eval(x))
# dataset['label'] = dataset['label'].map(label2id)

# with open('train_eval_idx.pkl', 'rb') as f:
#     train_eval_idx = pickle.load(f)
    
# train_dataset = dataset.iloc[train_eval_idx['train']]
# dev_dataset = dataset.iloc[train_eval_idx['eval']]

# train_label = np.array(label)[train_eval_idx['train']]
# dev_label = np.array(label)[train_eval_idx['eval']]

In [ ]:
class_labels = ClassLabel(num_classes=30, names=class_names)

hf_test_dataset = Dataset.from_pandas(dataset)

# hf_test_dataset = hf_test_dataset.remove_columns('__index_level_0__')

hf_test_dataset.features['label'] = ClassLabel(num_classes=30, names=class_names)

In [53]:
test_examples = hf_test_dataset.map(mark_entity_spans)

  0%|          | 0/7765 [00:00<?, ?ex/s]

In [54]:
tokenized_test_datasets = test_examples.map(convert_example_to_features)

  0%|          | 0/7765 [00:00<?, ?ex/s]

In [55]:
test_dataloader = DataLoader(tokenized_test_datasets, collate_fn=data_collator, batch_size=32, shuffle=False)

In [42]:
sample = next(iter(test_dataloader))

In [61]:
sample

{'input_ids': tensor([[    0,  3625,  3749,  ...,     1,     1,     1],
        [    0,  3784, 21154,  ...,     1,     1,     1],
        [    0,  3995,    16,  ...,  2062,    18,     2],
        ...,
        [    0,  1504,  2259,  ...,     1,     1,     1],
        [    0,  3753, 32000,  ...,     1,     1,     1],
        [    0,  9500,  2079,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100])}

In [56]:
model.eval()
output_pred = []
output_prob = []
for i, data in enumerate(tqdm(test_dataloader)):
    with torch.no_grad():
        outputs = model(
          input_ids=data['input_ids'].to(device),
          attention_mask=data['attention_mask'].to(device),
#           token_type_ids=data['token_type_ids'].to(device)
          )
    logits = outputs[0]
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)

    output_pred.append(result)
    output_prob.append(prob)

100%|██████████| 243/243 [00:56<00:00,  4.31it/s]


In [57]:
pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

In [58]:
def num_to_label(label):
  """
    숫자로 되어 있던 class를 원본 문자열 라벨로 변환 합니다.
  """
  origin_label = []
  with open('dict_num_to_label.pkl', 'rb') as f:
    dict_num_to_label = pickle.load(f)
  for v in label:
    origin_label.append(dict_num_to_label[v])
  
  return origin_label

In [59]:
pred_answer = num_to_label(pred_answer) # 숫자로 된 class를 원래 문자열 라벨로 변환.


In [63]:
output = pd.DataFrame({'id':dataset['id'].values,'pred_label':pred_answer,'probs':output_prob,})

In [64]:
output

,id,pred_label,probs
0,0,org:product,"[0.015178636647760868, 0.0011734288418665528, ..."
1,1,per:alternate_names,"[0.0028730828780680895, 0.00038902508094906807..."
2,2,no_relation,"[0.9991886019706726, 3.7351230275817215e-05, 3..."
3,3,no_relation,"[0.8227369785308838, 0.000364467705367133, 0.0..."
4,4,org:top_members/employees,"[0.13053059577941895, 0.8653919696807861, 0.00..."
...,...,...,...
7760,7760,no_relation,"[0.4702913761138916, 0.00040200879448093474, 0..."
7761,7761,no_relation,"[0.995478093624115, 0.000579211744479835, 0.00..."
7762,7762,org:top_members/employees,"[0.00033924198942258954, 0.9989251494407654, 2..."
7763,7763,no_relation,"[0.9862713813781738, 0.00011878733494086191, 5..."


In [66]:
output.to_csv('submission.csv', index=False)

In [ ]:
def inference(model, tokenized_sent, device):
  """
    test dataset을 DataLoader로 만들어 준 후,
    batch_size로 나눠 model이 예측 합니다.
  """
  dataloader = DataLoader(tokenized_sent, batch_size=16, shuffle=False)
  model.eval()
  output_pred = []
  output_prob = []
  for i, data in enumerate(tqdm(dataloader)):
    with torch.no_grad():
      outputs = model(
          input_ids=data['input_ids'].to(device),
          attention_mask=data['attention_mask'].to(device),
          token_type_ids=data['token_type_ids'].to(device)
          )
    logits = outputs[0]
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)

    output_pred.append(result)
    output_prob.append(prob)
  
  return np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

# Inference

# Others